<a href="https://colab.research.google.com/github/ishan13b/Pytorch-Implementation/blob/main/Feedforward_CIFAR100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [ ]:
torch.cuda.is_available()

True

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Hyper-parameters
input_size = 3072 # 32x32
hidden_size_1 = 500
hidden_size_2 = 100
num_classes = 100
num_epochs = 100
batch_size = 512
learning_rate = 0.01

In [ ]:
train_cifar100_data = torchvision.datasets.CIFAR100(root='/data', train=True, transform=transforms.ToTensor(), download=True)
test_cifar100_data = torchvision.datasets.CIFAR100(root='/data',train=False,transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(train_cifar100_data,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_cifar100_data,batch_size=batch_size,shuffle=False)


100%|██████████| 169M/169M [00:13<00:00, 12.6MB/s]


Extracting /data/cifar-100-python.tar.gz to /data


In [ ]:
images, label = train_cifar100_data[0]
print(images.shape)


torch.Size([3, 32, 32])


In [ ]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size_1, hidden_size_2, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size_1)
        self.l2 = nn.Linear(hidden_size_1, hidden_size_2)
        self.l3 = nn.Linear(hidden_size_2, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

model = NeuralNet(input_size, hidden_size_1,hidden_size_2, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
# Train the model
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [64, 3, 32, 32]
        # resized: [64, 3*32*32]
        images = images.reshape(-1, 3*32*32).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 3*32*32).to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')

Accuracy of the network on the 10000 test images: 16.2 %
